In [39]:
import argparse
import pickle
import re
from contextlib import ExitStack, nullcontext
from datetime import datetime, timedelta
from importlib import resources
from itertools import chain, tee
from os import environ, makedirs, path

 

import eccodes
import pdbufr
import numpy as np
import pandas as pd
from scipy.optimize import root_scalar
from scipy.spatial import KDTree

import warnings
warnings.filterwarnings("ignore")

In [85]:
# Function to import forecast storms file and load it in a dataframe
def create_storms_df():
    # Load cyclone dataframe with Mean sea level pressure value
    df_storms = pdbufr.read_bufr('track_data/tc_test_track_data.bufr',
        columns=("stormIdentifier", "ensembleMemberNumber", "year", "month", "day", "hour", "latitude", "longitude",
                 "pressureReducedToMeanSeaLevel"))
    # Load cyclone dataframe with Wind speed at 10m value
    df1 = pdbufr.read_bufr('track_data/tc_test_track_data.bufr',
        columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
                 "windSpeedAt10M"))
    # Load cyclone dataframe with timeperiod column
    df2 = pdbufr.read_bufr('track_data/tc_test_track_data.bufr',
        columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
                 "timePeriod"))
    # Add the Wind speed at 10m column to the storms dataframe 
    df_storms["windSpeedAt10M"] = df1.windSpeedAt10M
    df_storms["timePeriod"] = df2.timePeriod
    # Storms with number higher than 10 are not real storms (according to what Fernando said)
    drop_condition = df_storms.stormIdentifier < '11'
    df_storms = df_storms[drop_condition]
    return df_storms

df_storms = create_storms_df()

In [99]:
print(df_storms.stormIdentifier.unique())
df_storm = df_storms[df_storms.stormIdentifier == '07E']
df_storm.reset_index(drop=True, inplace=True)

['07E' '08E' '09E' '09W']


In [102]:
# Function to reorganize storm dataframe as input for strike probability map calculation

def storm_df_reorganization(df):
    df.rename(columns={"ensembleMemberNumber":"number", "latitude":"lat", "longitude":"lon", "pressureReducedToMeanSeaLevel":"msl", "windSpeedAt10M":"wind"}, inplace=True)
    df.drop(columns=['stormIdentifier'])
    df['msl'] = df_storm['msl'] / 100
    dates = datetime(df.year[0], df.month[0], df.day[0], df.hour[0]) + timedelta(hours=1) * df.timePeriod
    df["date"] = dates.dt.strftime("%Y%m%d")
    df["hours"] = dates.dt.strftime("%H").astype("int") * 100
    df.drop(columns=['year','month','day','hour'])
    df = df.reindex(['lat','lon','number','date','hours','wind','msl'], axis=1)
    return df

df_storm_re = storm_df_reorganization(df_storm)
df_storm_re.head()

,lat,lon,number,date,hours,wind,msl
0,16.6,-128.9,1,20230817,600,18.5,9.99
1,16.3,-130.1,1,20230817,1200,18.0,10.02
2,16.4,-131.3,1,20230817,1800,17.0,10.02
3,16.3,-132.4,1,20230818,0,16.0,10.05
4,16.4,-133.2,1,20230818,600,14.4,10.03


In [52]:
"%.03s" % dates[0].strftime("%H%M%M")

'060'

In [60]:
hours = dates.dt.strftime("%H").astype("int") * 100
hours

0        600
1       1200
2       1800
3          0
4        600
        ... 
1503     600
1504    1200
1505    1800
1506       0
1507     600
Name: timePeriod, Length: 1508, dtype: int64